In [1]:
import math
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from sklearn.utils import shuffle

In [70]:
from google.colab import drive
drive.mount('/content/drive')
gpu = 0 #gpu ID

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [147]:
def conv_branch_init(conv, branches):
    weight = conv.weight
    n = weight.size(0)
    k1 = weight.size(1)
    k2 = weight.size(2)
    nn.init.normal_(weight, 0, math.sqrt(2. / (n * k1 * k2 * branches)))
    nn.init.constant_(conv.bias, 0)


def conv_init(conv):
    nn.init.kaiming_normal_(conv.weight, mode='fan_out')
    nn.init.constant_(conv.bias, 0)


def bn_init(bn, scale):
    nn.init.constant_(bn.weight, scale)
    nn.init.constant_(bn.bias, 0)


        

# Temporal unit
class T_LSU(nn.Module):
    def __init__(self, in_channels, out_channels, T=16,V=20,return_seq=True):
        super(T_LSU, self).__init__()
        self.V=V
        self.out_channels = out_channels
        self.in_channels = in_channels
        self.return_seq = return_seq
        self.relu = nn.ReLU()
        self.lr = nn.Linear(out_channels*V, out_channels)
        self.lstm_layers=nn.ModuleList([nn.LSTM(in_channels,out_channels,num_layers=1,batch_first=True) for i in range(V)])
    def forward(self, x):
        # N, C, T, V
        N,C,T,V = x.size()
        x = x.permute(0,3,2,1).contiguous().view(N,V,T,C)
        output = torch.tensor([])
        if gpu is not None:
            output = output.cuda(gpu)
        assert C==self.in_channels
        for i in range(V):
            output_i,_ = self.lstm_layers[i](x[:,i,:,:])
            output_i = output_i.unsqueeze(1)
            if gpu is not None:
                output_i.cuda(gpu)
            output = torch.cat([output,output_i],dim=1)
        N,V,T,C = output.size()
        if self.return_seq:
            x = output.permute(0,3,2,1).contiguous().view(N,C,T,V)
            # N,C,T,V
            
            return x

        # N, V, C
        x = output[:,:,-1,:]
        x = x.contiguous().view(N,V*C)
        x = self.lr(x)
        return self.relu(x)

# Spatial unit
class S_LSU(nn.Module):
    def __init__(self, in_channels, out_channels, num_joints, num_heads=8, coff_embedding=4, bias=True):
        super(S_LSU, self).__init__()
        inter_channels = out_channels // coff_embedding
        self.inter_c = inter_channels
        self.out_channels = out_channels
        self.num_heads = num_heads
        self.DepM = nn.Parameter(torch.Tensor(num_heads, num_joints, num_joints))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(num_joints))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

        # Attention
        self.conv_a = nn.ModuleList()
        self.conv_b = nn.ModuleList()
        self.conv_d = nn.ModuleList()
        for i in range(self.num_heads):
            self.conv_a.append(nn.Conv2d(in_channels, inter_channels, 1))
            self.conv_b.append(nn.Conv2d(in_channels, inter_channels, 1))
            self.conv_d.append(nn.Conv2d(in_channels, out_channels, 1))

        if in_channels != out_channels:
            self.down = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.down = lambda x: x

        self.bn = nn.BatchNorm2d(out_channels)
        self.soft = nn.Softmax(-2)
        self.relu = nn.ReLU()

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                conv_init(m)
            elif isinstance(m, nn.BatchNorm2d):
                bn_init(m, 1)
        bn_init(self.bn, 1e-6)
        for i in range(self.num_heads):
            conv_branch_init(self.conv_d[i], self.num_heads)


    def reset_parameters(self) -> None:
        nn.init.kaiming_uniform_(self.DepM, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.DepM)
            bound = 1 / math.sqrt(fan_in)
            nn.init.uniform_(self.bias, -bound, bound)


    def forward(self, x):
        N, C, T, V = x.size()
        W = self.DepM
        B = self.bias
        y = None
        for i in range(self.num_heads):

            A1 = self.conv_a[i](x).permute(0, 3, 1, 2).contiguous().view(N, V, self.inter_c * T)
            A2 = self.conv_b[i](x).view(N, self.inter_c * T, V)
            A1 = self.soft(torch.matmul(A1, A2) / A1.size(-1))  # N tV tV

            A1 = W[i] + A1
            A2 = x.view(N, C * T, V)
            z = self.conv_d[i]((torch.matmul(A2, A1)).view(N, C, T, V))
            y = z + y if y is not None else z

        y = self.bn(y)
        y += self.down(x)
        return self.relu(y).view(N, -1, T, V)


class ST_block(nn.Module):
    def __init__(self, in_channels, out_channels, num_joints=25, num_heads=3,residual=True,return_seq=True):
        super(ST_block, self).__init__()
        self.s_unit = S_LSU(in_channels, out_channels, num_joints, num_heads)
        self.t_unit = T_LSU(out_channels, out_channels,return_seq=return_seq)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
    
        if residual:
            self.residual = lambda x: x
        else:
            self.residual = lambda x: 0

    def forward(self, x):
        s_out = self.dropout(self.s_unit(x))
        x = self.dropout(self.t_unit(s_out) + self.residual(s_out))
        return self.relu(x)


class UNIK(nn.Module):
    def __init__(self, num_class=49, num_joints=20, num_heads=3, in_channels=3):
        super(UNIK, self).__init__()

        self.data_bn = nn.BatchNorm1d(in_channels * num_joints)

        self.l1 = ST_block(in_channels, 64, num_joints, residual=False)
        self.l2 = ST_block(64, 128, num_joints, num_heads)
        self.l3 = ST_block(128, 256, num_joints, num_heads)
        self.l4 = ST_block(256, 128, num_joints, num_heads, return_seq = False,residual=False)
        self.fc = nn.Linear(128, num_class)
        nn.init.normal_(self.fc.weight, 0, math.sqrt(2. / num_class))
        bn_init(self.data_bn, 1)

    def forward(self, x):
        N, C, V, T = x.size()
        x = x.permute(0,2,1,3).contiguous().view(N,V*C,T)
        x = self.data_bn(x)
        x = x.view(N,V,C,T).permute(0,2,3,1).contiguous().view(N,C,T,V)

        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        x = self.l4(x)
        x=x.view(N,-1)

        return self.fc(x)

In [4]:
class GCNDataset(Dataset):
    def __init__(self,filename,hasLabel=True,aug=False):
        self.df = pd.read_csv(filename,header=None)
        self.length = len(self.df)
        if hasLabel:
            #self.df['freq']=1./self.df.groupby(961)[961].transform('count')
            #self.df = self.df.sample(frac=1,weights=self.df.freq).reset_index(drop=True)
            self.df = self.df.sample(frac=1).reset_index(drop=True)
            self.X = torch.tensor(self.df.iloc[:,1:-1].values.astype('float32'))
            # N, C, V, T
            self.X = self.X.reshape((self.length ,16, 20, 3)).permute(0,3,2,1).contiguous()
            if aug:
                X1=self.X
                X2=self.X
                X3=self.X
                X1[:,0,:,:] *= -1
                X2[:,1,:,:] *= -1
                X3[:,2,:,:] *= -1
                self.X = torch.cat((X1,X2,X3,self.X),0)
            self.labels = self.df.iloc[:,-1].values.astype('int32')-1
            self.Y = torch.tensor(self.labels,dtype=torch.long)
        else:
            self.X = torch.tensor(self.df.iloc[:,1:].values.astype('float32'))
            self.X = self.X.reshape((self.length ,16, 20, 3)).permute(0,3,2,1).contiguous()
            self.Y=torch.tensor(np.zeros(self.length),dtype=torch.long)

    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,index):
        x = self.X[index]
        y = self.Y[index]
        return x, y 

In [152]:
root_path = "drive/MyDrive/SML/"
bs=64
train_set = GCNDataset(filename=root_path+"training_set.csv",aug=False)
val_set = GCNDataset(filename=root_path+"val_set.csv")
test_set = GCNDataset(filename=root_path+"test.csv",hasLabel=False)
train_loader = DataLoader(train_set,batch_size=bs,num_workers=2,shuffle=True)
val_loader = DataLoader(val_set,batch_size=bs)
test_loader = DataLoader(test_set,batch_size=bs)

In [153]:
net = UNIK()
net.load_state_dict(torch.load(root_path+'UNIK_r_best.pkl'))
criterion = nn.CrossEntropyLoss()

net.cuda(gpu)
#opti = optim.SGD(net.parameters(),lr=0.01, weight_decay=0.0005)
opti = optim.Adam(net.parameters(),lr=0.01,weight_decay=0.0005)
opti.load_state_dict(torch.load(root_path+'UNIK_r_optim.pkl'))
print()

In [154]:
def adjust_learning_rate(lr,wd):
    for param_group in opti.param_groups:
        param_group['lr'] = lr
    return lr
adjust_learning_rate(5e-5,0.0002)

def accuracy(logit,target):
    a=(torch.argmax(logit,dim=1)==target).sum()
    return a

def evaluate(model, criterion, dataloader, gpu):
    model.eval()
    acc = 0
    count = 0
    with torch.no_grad():
        for i,(x,y) in enumerate(dataloader):
            x,y = x.cuda(gpu), y.cuda(gpu)
            logits = model(x)
            acc+= accuracy(logits, y)
            count += bs

    return acc / count

def train():
    best_acc=0.494
    best_epoch = 0
    for epoch in range(200):
        total = 0
        correct = 0
        """
        if epoch==0:
            adjust_learning_rate(0.01)
        elif epoch==20:
            adjust_learning_rate(0.001)
        elif epoch==40:
            adjust_learning_rate(0.0001)
        elif epoch==60:
            adjust_learning_rate(0.00001)
        """
        for i, (x,y) in enumerate(train_loader):
            net.train()
            opti.zero_grad()
            x,y=x.cuda(gpu),y.cuda(gpu)
            logit = net(x)
            loss = criterion(logit,y)
            loss.backward()
            opti.step()
            correct+=accuracy(logit,y)
            total+=bs
        dev_acc = evaluate(net, criterion, val_loader, gpu)
        
        if dev_acc>best_acc:
            best_acc=dev_acc
            torch.save(net.state_dict(), root_path+'UNIK_r_best.pkl')
            torch.save(opti.state_dict(), root_path+"UNIK_r_optim.pkl")
        print("epoch",epoch,"train acc:",round(float(correct/total),5),"dev_acc:",round(float(dev_acc),5))
train()

epoch 0 train acc: 0.66155 dev_acc: 0.48646
epoch 1 train acc: 0.66274 dev_acc: 0.48646
epoch 2 train acc: 0.66817 dev_acc: 0.4875
epoch 3 train acc: 0.66565 dev_acc: 0.48594
epoch 4 train acc: 0.67505 dev_acc: 0.48333
epoch 5 train acc: 0.67293 dev_acc: 0.48542


KeyboardInterrupt: ignored

In [ ]:
def predict(net,dataloader):
    net.eval()
    predictions = torch.tensor([]).cuda(gpu)
    with torch.no_grad():
        for i, (x,y) in enumerate(test_loader):
            x,y=x.cuda(gpu),y.cuda(gpu)
            logit = net(x)
            pred = torch.argmax(logit,dim=1)+1
            predictions=torch.cat((predictions,pred))
    return predictions.cpu().numpy()
pred = predict(net,test_loader)
print(pred)

In [ ]:
output = pd.read_csv(root_path+"sample.csv")
output['Category']=pred.astype('int')
output.head()
output.to_csv("predictions.csv",index=None)